In [1]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, precision_recall_curve
import time
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import StandardScaler

In [3]:
INPUT = os.getcwd()
INPUT = os.path.join(INPUT, "points_norm")
INPUT

'/home/superusuario/Dados/Mapbiomas/col6_0/points_norm'

In [4]:
ls_bacia_ruins = [
    '744','746','751','753','754','756','757',
    '758','763','765','766','767','771','772',
    '776','7421','7422','7612','7613','7614',
    '7615','7619', '7741','7742'
]

In [5]:
allfiles = glob.glob(os.path.join(INPUT, "*.csv"))
df_from_each_file = []
zz = 0
for cc, file in enumerate(allfiles):  
    ls_parte = file.split('/')
    name_bacia = ls_parte[-1].replace('_col6.csv', '')
    if name_bacia in ls_bacia_ruins:
        zz += 1
        print('lindo {} de {} ==> {}'.format(zz, cc ,file))
        newdf = pd.read_csv(file)     
        newdf = newdf.drop(['system:index','.geo'], axis=1)
        df_from_each_file.append(newdf)
        
print("leiu {} de {} no folder".format(zz, len(allfiles)))
        

lindo 1 de 0 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/753_col6.csv
lindo 2 de 3 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/765_col6.csv
lindo 3 de 4 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/7619_col6.csv
lindo 4 de 7 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/746_col6.csv
lindo 5 de 10 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/7421_col6.csv
lindo 6 de 13 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/771_col6.csv
lindo 7 de 15 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/7742_col6.csv
lindo 8 de 16 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/7613_col6.csv
lindo 9 de 17 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/7422_col6.csv
lindo 10 de 18 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/776_col6.csv
lindo 11 de 19 ==> /home/superusuario/Dados/Mapbiomas/col6_0/points_norm/7614_col6.csv
lindo 12 de 21 ==> /home/superusuario/Dados/Mapbiomas/col6_0/

In [6]:
concat_df  = pd.concat(df_from_each_file, axis=0, ignore_index=True)
print("temos {} filas ".format(concat_df.shape))

temos (5318374, 88) filas 


In [7]:
concat_df.head()

,awei,awei_median,awei_median_dry,bacia,blue,blue_1,blue_median,blue_median_dry,blue_stdDev,brightness,...,swir2,swir2_1,swir2_median,swir2_median_dry,swir2_stdDev,versao,wetness,wetness_median,wetness_median_dry,year
0,0.0,0.0,0.0,753,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5,0.0,0.0,0.0,1991
1,0.0,0.0,0.0,753,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5,0.0,0.0,0.0,1991
2,0.0,0.0,0.0,753,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5,0.0,0.0,0.0,1991
3,0.0,0.0,0.0,753,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5,0.0,0.0,0.0,1991
4,0.0,0.0,0.0,753,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5,0.0,0.0,0.0,1991


In [ ]:
columns_features = [
           'blue_median', 'blue_median_dry', 'blue_stdDev', # 'blue', 'blue_1',
           'green_median', 'green_median_dry', 'green_stdDev', # 'green', 'green_1',
           'red_median', 'red_median_dry','red_stdDev',  # 'red', 'red_1', 
           'nir_median','nir_median_dry', 'nir_stdDev', #'nir', 'nir_1', 
           'swir1_median','swir1_median_dry', 'swir1_stdDev',  # 'swir1', 'swir1_1', 
           'swir2_median','swir2_median_dry', 'swir2_stdDev', # 'swir2', 'swir2_1', 
           'ndwi','ndwi_median', 'ndwi_median_dry',
           'iia', 'iia_median', 'iia_median_dry', 
           'awei', 'awei_median', 'awei_median_dry',
           'ndvi', 'ndvi_median', 'ndvi_median_dry',
           'evi', 'evi_median', 'evi_median_dry',
           'cvi','cvi_median', 'cvi_median_dry', 
           'lai','lai_median', 'lai_median_dry',
           'osavi', 'osavi_median', 'osavi_median_dry',
           'rvi', 'rvi_median', 'rvi_median_dry',
           'gcvi', 'gcvi_median', 'gcvi_median_dry',
           'brightness','brightness_median', 'brightness_median_dry',
           'ratio', 'ratio_median','ratio_median_dry',
           'spri', 'spri_median','spri_median_dry',
           'wetness','wetness_median', 'wetness_median_dry',
           'npv', 'npv_median', 'npv_median_dry',
           'ndfia', 'ndfia_median', 'ndfia_median_dry',
           'soil','soil_median', 'soil_median_dry',
           'gv', 'gv_median','gv_median_dry'           
        ]
classe = "class"
featureTest = ['blue', 'blue_1','blue_median', 'blue_median_dry', 'blue_stdDev']
featureTest2 = ['iia', 'iia_median', 'iia_median_dry']

In [ ]:
len(columns_features)

In [ ]:
corr = concat_df[columns_features].corr()
plt.figure(figsize=(40,40))
sns.heatmap(corr, cmap="plasma_r",center=0, annot=True)

## remove features with correlation higher than 0.8

In [ ]:
size_matrix = corr.shape[0]
columns = np.full((size_matrix), True, dtype= bool)
for row in range(size_matrix):
    for col in range(row + 1, size_matrix):
        if corr.iloc[row,col] >= 0.9:
            if columns[col]:
                columns[col] = False
print(columns)
print()

In [ ]:
print(columns.shape)
print(type(columns))
columns = pd.Series(columns)
print(type(columns))

In [ ]:
selected_columns = concat_df[columns_features].columns[columns]
selected_columns = [kk for kk in selected_columns]
print(selected_columns)
print("with {} Features".format(len(selected_columns)))

In [ ]:
# ls_bnd = [
#            'blue_median', 'blue_median_dry', 'blue_stdDev', # 'blue', 'blue_1',
#            'green_median', 'green_median_dry', 'green_stdDev', # 'green', 'green_1',
#            'red_median', 'red_median_dry','red_stdDev',  # 'red', 'red_1', 
#            'nir_median','nir_median_dry', 'nir_stdDev', #'nir', 'nir_1', 
#            'swir1_median','swir1_median_dry', 'swir1_stdDev',  # 'swir1', 'swir1_1', 
#            'swir2_median','swir2_median_dry', 'swir2_stdDev', # 'swir2', 'swir2_1', 
# ]
# selected_columns = selected_columns + ls_bnd
# # selected_columns
# print("And now    with {} Features".format(len(selected_columns)))

In [7]:
selected_columns = [
    'blue_median', 'blue_median_dry', 'blue_stdDev', 'red_stdDev', 
    'red_median','nir_median', 'nir_stdDev', 'swir1_median', 'swir1_stdDev', 
    'swir2_stdDev', 'ndwi', 'iia', 'ndvi', 'cvi', 'cvi_median', 
    'cvi_median_dry', 'rvi', 'rvi_median', 'rvi_median_dry', 
    'gcvi', 'ratio_median', 'npv', 'npv_median', 'npv_median_dry', 
    'ndfia', 'ndfia_median', 'ndfia_median_dry', 'soil', 'soil_median', 
    'soil_median_dry', 'gv', 'gv_median', 'gv_median_dry'
]

In [ ]:
corr_red = concat_df[selected_columns].corr()
plt.figure(figsize=(40,40))
sns.heatmap(corr_red, cmap="plasma_r", center=0, annot=True)

In [ ]:
cg = sns.clustermap(corr_red, cmap ="YlGnBu", linewidths = 0.1)
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation = 0)

In [ ]:
m_classe = [kk for kk in concat_df['class'].unique()]
m_classe

In [ ]:
sorted(m_classe)

In [ ]:
label_class = ['florestal','savana', 'campo', 'agri_past', 'area_nao_veg', 'água']
ls_color = ['#006400','#32CD32','#B8AF4F', "#FFEFC3", "EA9999", "0000FF"]
ls_name_color = ['darkgreen','green','khaki','wheat','ivory','blue']

In [ ]:
fig = plt.figure(figsize = (20, 25))
j = 0
for feat in selected_columns:
    plt.subplot(6, 4, j+1)    
    j += 1
#     ls_columns = [feat, 'class']
    for cc, cclas in enumerate(m_classe):  
        
        sns.distplot(concat_df[concat_df['class']== cclas][feat]) #color= ls_name_color[cc],,  label = label_class[cc]
        
        plt.legend(loc='best')
fig.suptitle('Breast Cance Data Analysis')
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

In [8]:
ls_1 = ['blue_median', 'blue_stdDev', 'nir_median']  # , 'nir_stdDev'
ls_2 = ['swir1_median', 'swir1_stdDev', 'ndwi', 'iia']
ls_3 = ['cvi', 'cvi_median','cvi_median_dry', 'npv']
ls_4 = ['npv_median', 'npv_median_dry', 'ndfia','ndfia_median']
ls_5 = ['ndfia_median_dry', 'soil', 'soil_median','soil_median_dry']
ls_6 = ['gv', 'gv_median', 'gv_median_dry']

In [9]:
sns.pairplot(vars=ls_1, data= concat_df, hue='class', height=3) #height=5, markers=["o", "h",'>','<','^','p']
sns.set_context("notebook", font_scale=1.5)  

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f5a25f51160> (for post_execute):


KeyboardInterrupt: 

In [ ]:
# sns.pairplot(vars=['B11_d', 'B4_d', 'awei_d', 'B11', 'wetness_d'], data= concat_df, hue='class', height=5, markers=["o", "s"])
# sns.set_context("notebook", font_scale=1.5) 

In [ ]:
sns.pairplot(vars=ls_2, data= concat_df, hue='class', height=5, markers=["o", "v",'>','<','^','.'])
sns.set_context("notebook", font_scale=1.5)  

In [ ]:
sns.pairplot(vars=ls_3, data= concat_df, hue='class', height=5, markers=["o", "v",'>','<','^','.'])
sns.set_context("notebook", font_scale=1.5)  

In [ ]:
sns.pairplot(vars=ls_4, data= concat_df, hue='class', height=5, markers=["o", "v",'>','<','^','.'])
sns.set_context("notebook", font_scale=1.5)  

In [ ]:
sns.pairplot(vars=ls_5, data= concat_df, hue='class', height=5, markers=["o", "v",'>','<','^','.'])
sns.set_context("notebook", font_scale=1.5)  

In [ ]:
sns.pairplot(vars=ls_6, data= concat_df, hue='class', height=5, markers=["o", "v",'>','<','^','.'])
sns.set_context("notebook", font_scale=1.5)  

## Selecting columns based on p-values
selected_columns

In [ ]:
import statmodels.formula.api as sm

In [ ]:
selected_solumns = selected_columns[1:].values
def backwardElimination(x, Y, sl, columns):
    
    numVars = len(x[0])
    
    for i in range(numVars):
        
        regressor_OLS = sm.OLS(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(numVars - i):
                if regressor_OLS.pvalues[j].astype(float) == maxVar:
                    x = np.delete(x, j, 1)
                    columns = np.delete(columns, j)
                    


In [ ]:
SL = 0.05
data_modeled, selected_columns = backwardElimination(
                            concat_df[selected_columns][:, 1:].values, 
                            concat_df[selected_columns][:, 1:].values, 
                            SL, selected_columns)

In [ ]:
selected_columns = ['blue_median', 'blue_stdDev', 'nir_median', 'nir_stdDev',
       'swir1_median', 'swir1_stdDev', 'ndwi', 'iia', 'cvi', 'cvi_median',
       'cvi_median_dry', 'npv', 'npv_median', 'npv_median_dry', 'ndfia',
       'ndfia_median', 'ndfia_median_dry', 'soil', 'soil_median',
       'soil_median_dry', 'gv', 'gv_median', 'gv_median_dry']

## Modelos para classificação
#Passos:
1. Criar base de dados de teste e treinamento
2. Instanciar o modelo
3. Especificar hiperparâmetros
4. Instanciar GridSearchCV e RandomizedSearchCV 
5. Realizar treinamento na base de dados de treino
6. Realizar predict
7. Avaliar modelo com a base de dados de teste

In [ ]:
X = concat_df[selected_columns]
y = concat_df[classe]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [ ]:
metrics = pd.DataFrame(index=['accuracy', 'precision', 'recall', 'AUC', 'Time to Train'], 
                       columns=['Random Forest'])
metrics

In [ ]:
classifier_rf = RandomForestClassifier(random_state=0)

In [ ]:
param_grid_rf = {
    'classifier__n_estimators': [70, 100, 130, 150, 170],
    'classifier__criterion': ["entropy"],
    'classifier__max_depth': [3, 5, 10, None],
    'classifier__min_samples_leaf': randint(1, 9),
}

In [ ]:
param_grids = [param_grid_rf]
model_names = ['Random Forest']
cv_names = ['rf_cv']
cvs = [5]

In [ ]:
def train_and_score_model(model_name, estimator, param_grid=None, cv=5, iid=False):
    
    #### Step 5: Instantiate the CV Object
    
    if model_name == 'Logistic Regression':
        cv = GridSearchCV(estimator, param_grid, cv=cv,iid=False,)  # , iid=False
    else: 
        cv = RandomizedSearchCV(estimator, param_grid, cv=cv,  random_state=None)

    # Treinamento
    t0 = time.time()
    cv.fit(x_train, y_train)
    ttt = time.time() - t0 # ttt stands for "time to train"
    print("It takes %.3f seconds for %s fitting" % (ttt, model_name))
    metrics.loc['Time to Train', model_name] = ttt

    #predict

    y_pred = cv.predict(x_test)

    # Avaliação

    print("Accuracy with " + model_name + "on the testing set is: %.3f" % cv.score(x_test, y_test))

    metrics.loc['accuracy', model_name] = accuracy_score(y_pred=y_pred, y_true=y_test)
    metrics.loc['precision', model_name] = precision_score(y_pred=y_pred, y_true=y_test, average='micro')
    metrics.loc['recall', model_name] = recall_score(y_pred=y_pred, y_true=y_test, average='micro')
    
    '''
    if model_name != 'Support Vector Machine':
        y_pred_prob = cv.predict_proba(x_test)[:,1]
        metrics.loc['AUC', model_name] = roc_auc_score(y_test, y_pred_prob)
    '''

    #print(Conf_Matrix(confusion_matrix(y_test, y_pred)))
    print(classification_report(y_test, y_pred))

    '''
    if model_name != 'Support Vector Machine':
        # ROC Curve
        fpr, tpr, thresholds  = roc_curve(y_test, y_pred_prob, multi_class="ovr")
        plt.plot([0,1], [0,1], 'k--')
        plt.plot(fpr, tpr, label=model_name)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('%s ROC Curve' % model_name)
        plt.show()
    '''
    return cv

#### Anális das features para uma bacia

In [ ]:
pipeline = Pipeline(steps=[('classifier', classifier_rf)])
rf_cv = train_and_score_model('Random Forest', pipeline, param_grid_rf)

In [ ]:
rf_cv.best_params_

In [ ]:
classifier_rf = RandomForestClassifier(criterion='entropy', min_samples_leaf=1, n_estimators=100, random_state=0)
score = classifier_rf.fit(x_train, y_train)

In [ ]:
importances = pd.Series(data=classifier_rf.feature_importances_, index=columns_features)
importances = importances.sort_values(ascending = False)
plt.figure(figsize=(10,25))
sns.barplot(x=importances, y=importances.index, orient='h').set_title('Importância de cada feature')

#### Anaálise das fatures para somente uma bacia

In [ ]:
pipeline = Pipeline(steps=[('classifier', classifier_rf)])
rf_cv = train_and_score_model('Random Forest', pipeline, param_grid_rf)

In [ ]:
rf_cv.best_params_

In [ ]:
classifier_rf = RandomForestClassifier(criterion='entropy', min_samples_leaf=3, n_estimators=70, random_state=0)
score = classifier_rf.fit(x_train, y_train)

In [ ]:
importances = pd.Series(data=classifier_rf.feature_importances_, index=features)
importances = importances.sort_values(ascending = False)
plt.figure(figsize=(10,25))
sns.barplot(x=importances, y=importances.index, orient='h').set_title('Importância de cada feature')

Análise das melhores features para a Caatinga

In [ ]:
features = importances.axes
features = np.array(features)
features = features[0]
features = features[:12]
classe = "class"

x = dados[features]
y = dados[classe]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

classifier_rf = RandomForestClassifier(criterion='entropy', min_samples_leaf=4, n_estimators=50, random_state=0)

classifier_rf.fit(x_train, y_train)

y_pred = classifier_rf.predict(x_test)

accuracy_score(y_pred=y_pred, y_true=y_test)

In [ ]:
x_rfc = dados[features]
y_rfc = dados[classe]
features_selecteds = []

x_train_rfc, x_test_rfc, y_train_rfc, y_test_rfc = train_test_split(x_rfc, y_rfc, test_size=0.4)

for k in range(1,12):
  rfe = RFE(classifier_rf, k)
  fit = rfe.fit(x_train_rfc, y_train_rfc)

  cols = fit.get_support(indices=True)

  for i in cols:
    features_selecteds.append(features[i])

  print("features: ", features_selecteds)
  x = dados[features_selecteds]
  y = dados[classe]

  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

  classifier_rf.fit(x_train, y_train)
  y_pred = classifier_rf.predict(x_test)
  acc = accuracy_score(y_pred=y_pred, y_true=y_test)

  print("acc com ", k, "features:  ", acc)
  features_selecteds = []
  
  

Análise da separabilidade das melhores Features

In [ ]:
inputs_selec= ['min_red', 'median_gcvi', 'median_swir2_dry', 'median_hallcover',
       'median_swir1_dry', 'median_swir2', 'median_gcvi_wet',
       'median_sefi', 'median_red_dry', 'class']
dadisAnalise = dados[inputs_selec]
dadisAnalise = dadisAnalise[:1000]

sns.pairplot(dadisAnalise, hue="class", height=2.5, corner=True)

Classifcação por bacia
Classificação por ano
Coleetar para a série anterior de mosaico
Guardar lsita d feeature das 30 primeiroas ordenada por bacia e por ano
Mudar paleta de cores